In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from net import CNN
from mydataset import MyDataSet
from myloss import TripletLoss

torch.manual_seed(10)

In [2]:
EPOCH = 25
BATCH_SIZE = 4
LR = 0.001

In [3]:
transform = transforms.Compose([   
        transforms.Resize([96, 96]),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=(0,0,0),std=(1,1,1))
])

train_set = MyDataSet('./birds/train/',transform=transform, labels=None)
train_loader = Data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_set = MyDataSet('./birds/test/', transform=transform, labels=None)
test_loader = Data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [4]:
print(train_set.classes)
print(train_set.class_to_idx)
print(train_set.__len__)

print(test_set.classes)
print(test_set.class_to_idx)
print(test_set.__len__)

['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird']
{'001.Black_footed_Albatross': 0, '002.Laysan_Albatross': 1, '003.Sooty_Albatross': 2, '004.Groove_billed_Ani': 3, '005.Crested_Auklet': 4, '006.Least_Auklet': 5, '007.Parakeet_Auklet': 6, '008.Rhinoceros_Auklet': 7, '009.Brewer_Blackbird': 8, '010.Red_winged_Blackbird': 9}
<bound method MyDataSet.__len__ of <mydataset.MyDataSet object at 0x000001E72171E8D0>>
['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird']
{'001.Black_footed_Albatross': 0, '002.Laysan_Albatross': 1, '003.Sooty_Albatross': 2, '004.Groove_billed_Ani': 3, '005.Crested_Auklet': 4, '006.Lea

In [5]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=18432, out_features=768, bias=True)
  (out): Linear(in_features=768, out_features=10, bias=True)
)


In [9]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
cross_loss = nn.CrossEntropyLoss()   # the target label is not one-hotted
triplet_loss = TripletLoss(0.5) # 选择损失函数
alpha = 0.5

for epoch in range(EPOCH):
    print('EPOCH ' + str(epoch))
    # for step, (b_x, b_y) in enumerate(train_loader):
    for step, (anchor, positive, negative) in enumerate(train_loader):
        #output = cnn(b_x)[0]
        #loss = loss_func(output, b_y)
        anchor_output = cnn(anchor[0])
        positive_output = cnn(positive[0])
        negative_output = cnn(negative[0])
        
        #print(anchor_output.detach().numpy().shape)   
        #output = torch.stack((anchor_output[0],  positive_output[0], negative_output[0]), dim=0)
        #b_y = torch.stack((anchor[1], positive[1], negative[1]), dim=0)
        #print(output)
        print(anchor_output[0])
        #b_y = torch.zeros(4, 10).scatter_(1, anchor[1].unsqueeze(1), 1).long()
        #print(b_y)
        
        
        triplet_result = triplet_loss(anchor_output[1], positive_output[1], negative_output[1])
        cross_result = cross_loss(anchor_output[0], anchor[1])
        loss = alpha * triplet_result + (1-alpha) * cross_result
                                       
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            print('训练三元损失：', triplet_result)
            print('训练交叉熵损失：', cross_result)
            
            correct = 0
            total = 0
            
            for _, (test, _, _) in enumerate(test_loader):
                test_output = cnn(test[0])[0]
                predicted = torch.max(test_output, 1).indices
                labels = test[1]
                total += labels.size(0)
                correct += (predicted == labels).sum()
                
            print('Accuracy on the test images: %d %%' % (100 * correct / total))
            
#             for data in test_loader:
#                 images,labels = data
#                 outputs = cnn(Variable(images))
#                 predicted = torch.max(outputs[0], 1).indices
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum()
                
#             print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
print('End')

EPOCH 0
tensor([[ 0.9950,  0.8326,  0.8629, -1.2571, -0.5171, -0.1255, -0.4228,  0.1251,
         -0.1411, -0.6718],
        [ 1.2372,  1.1303, -0.0778, -0.4902, -0.5986, -0.1048, -1.7202,  1.5874,
          0.6583, -1.8974],
        [-2.6336, -0.8540, -0.9405,  3.3358,  0.4838, -1.1074,  0.3242, -1.3594,
          1.0657,  1.4387],
        [-2.3256, -2.8093,  0.3071,  1.0234,  0.4051, -4.8028, -2.3147, -0.3483,
          3.0048,  5.0217]], grad_fn=<AddmmBackward>)
训练三元损失： tensor(0.4634, grad_fn=<MeanBackward0>)
训练交叉熵损失： tensor(0.9331, grad_fn=<NllLossBackward>)


KeyboardInterrupt: 